# KPI1 Identification des acteurs les plus présents et les périodes associées
# KPI2 : Evolution de la durée moyenne des films au fil des années
# KPI3 : Comparaison entre les acteurs présents au cinéma et dans les séries
# KPI4 : Age moyen des acteurs.
# KPI5 : Les films les mieux notés et les caractéristiques qu’ils partagent.

## Pour répondre à ces KPI il me faut un filtre sur les tconst de films et et séries qu'il est plus simple de faire sur vscode:
### Après exploration: films => titleType = movie, episode série => titleType = tvEpisode


In [2]:
import pandas as pd

In [3]:
df_t_basics = pd.read_csv("./BD/df_t_basics_clean.csv")

In [55]:
df_basic_name_pbi = pd.read_csv("./BD/df_basic_name_clean.csv")

In [46]:
df_t_ratings = pd.read_csv("./BD/df_t_ratings_clean.csv")

In [64]:
df_t_principals = pd.read_csv("./BD/df_t_principals_clean.csv")

In [34]:
#On filtre les ID pour ne garder que les films et episodes de séries non pornographiques
df_t_basics_pbi = df_t_basics[((df_t_basics['titleType'] == 'movie') | (df_t_basics['titleType'] == 'tvEpisode')) & (df_t_basics['isAdult'] == 0)]

#On ajoute les notes de la table ratings
info_films = pd.merge(df_t_basics_pbi, df_t_ratings, how = 'left', on = 'tconst')

#On sélectionne les colonnes pertinentes
info_films= info_films[['tconst', 'titleType', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes']]

#Export pour bi
info_films.to_csv("info_films.csv", index = False)

In [ ]:
#On enlèves les colonnes que nous n'utiliseront pas
info_acteurs = df_basic_name_pbi.drop(['knownForTitles', 'deathYear'], axis = 1)

#On ne garde que les acteurs et actrices
info_acteurs = info_acteurs[(info_acteurs[('primaryProfession')].str.contains('actor')) | (info_acteurs[('primaryProfession')].str.contains('actress'))]

#On signale si c'est un acteur ou une actrice avant de droper les autres professions
info_acteurs['profession'] = info_acteurs['primaryProfession'].apply(lambda x : 'actor' if 'actor' in x else 'actress')
info_acteurs = info_acteurs.drop('primaryProfession', axis =1)

#Export
info_acteurs.to_csv("info_acteurs.csv", index = False)

In [78]:
#On prend les id de films qui nous intéressent : ils sont déjà triés donc on ne garde que les colonnes id et startYear:
tconst = df_t_basics_pbi[['tconst', 'startYear']]

#On ajoute les id des gens qui ont bossés sur le film:
info_liens = pd.merge(tconst, df_t_principals, how = 'left', on = 'tconst')

#On reprend les acteurs et leur date de naissance:
act_id = info_acteurs[['nconst', 'birthYear']]

#On ajoute les dates de naissance aux id correspondants:
info_liens= pd.merge(info_liens, act_id, how = 'left', on ='nconst')

#Les nconst qui n'ont pas de date de naissance ne sont pas des acteurs : on peut drop les lignes!
info_liens =info_liens.dropna(subset = 'birthYear')
info_liens =info_liens.drop('category', axis = 1)

#On créé l'age de l'acteur au moment du film puis on supprime les autres colonnes années:
info_liens['age_act'] = info_liens['startYear'] -info_liens['birthYear']
info_liens = info_liens.drop(['startYear', 'birthYear'], axis = 1)

info_liens = info_liens.reset_index().drop('index', axis = 1)

#Export
info_liens.to_csv("info_liens.csv", index = False)
